In [5]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.backend import clear_session
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import datasets
import optuna

In [11]:
N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 10
def objective(trial):
    # Clear clutter from previous tf.keras session graphs.
    clear_session()
    (x_train, y_train), (x_valid, y_valid) = datasets.fashion_mnist.load_data()
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)[:N_TRAIN_EXAMPLES].astype("float32") / 255
    x_valid = x_valid.reshape(-1, img_x, img_y, 1)[:N_VALID_EXAMPLES].astype("float32") / 255
    y_train = y_train[:N_TRAIN_EXAMPLES]
    y_valid = y_valid[:N_VALID_EXAMPLES]
    input_shape = (img_x, img_y, 1)

    model = models.Sequential()
    model.add(
        layers.Conv2D(
            filters=trial.suggest_categorical("filters", [32, 64]),
            kernel_size=trial.suggest_categorical("kernel_size", [3, 5]),
            strides=trial.suggest_categorical("strides", [1, 2]),
            activation=trial.suggest_categorical("activation", ["relu", "linear"]),
            input_shape=input_shape,
        )
    )
    model.add(layers.Flatten())
    model.add(layers.Dense(CLASSES, activation="softmax"))

    # We compile our model with a sampled learning rate.
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizers.RMSprop(lr=lr), metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-03-01 15:20:16,041] A new study created in memory with name: no-name-9563d9cb-84c5-43ed-83a0-cf65f86f4cfe
[I 2021-03-01 15:20:26,562] Trial 0 finished with value: 0.6880000233650208 and parameters: {'filters': 32, 'kernel_size': 3, 'strides': 1, 'activation': 'linear', 'lr': 2.3247968244705566e-05}. Best is trial 0 with value: 0.6880000233650208.
[I 2021-03-01 15:20:43,083] Trial 1 finished with value: 0.6499999761581421 and parameters: {'filters': 64, 'kernel_size': 3, 'strides': 1, 'activation': 'relu', 'lr': 1.2463826884762187e-05}. Best is trial 0 with value: 0.6880000233650208.
[I 2021-03-01 15:20:47,594] Trial 2 finished with value: 0.7459999918937683 and parameters: {'filters': 32, 'kernel_size': 5, 'strides': 2, 'activation': 'linear', 'lr': 0.00016546289746074633}. Best is trial 2 with value: 0.7459999918937683.
[I 2021-03-01 15:20:53,408] Trial 3 finished with value: 0.8489999771118164 and parameters: {'filters': 64, 'kernel_size': 3, 'strides': 2, 'activation': 'relu

Number of finished trials: 94
Best trial:
  Value: 0.8550000190734863
  Params: 
    filters: 64
    kernel_size: 3
    strides: 2
    activation: relu
    lr: 0.0021793455046968243
